# Track filtering/fitting with LSTMs

This is a continuous space model using the ACTS data.

This should eventually move into a more appropriate folder. It's just here now for convenience.

In [1]:
# Select a GPU first
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '6'
os.environ['OMP_NUM_THREADS'] = '4'
cuda = False

In [2]:
from __future__ import print_function

# System imports
from timeit import default_timer as timer

# Data libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Torch imports
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

# Visualization
import matplotlib.pyplot as plt

# Local imports
from data import process_files
import torchutils
torchutils.set_cuda(cuda)
from torchutils import np_to_torch, torch_zeros, torch_to_np
from estimator import Estimator

# Magic
%matplotlib notebook

## Utilities

In [3]:
def select_hits(hits):
    # Select all barrel hits
    vids = [8, 13, 17]
    barrel_hits = hits[np.logical_or.reduce([hits.volid == v for v in vids])]
    # Re-enumerate the volume and layer numbers for convenience
    volume = pd.Series(-1, index=barrel_hits.index, dtype=np.int8)
    vid_groups = barrel_hits.groupby('volid')
    for i, v in enumerate(vids):
        volume[vid_groups.get_group(v).index] = i
    # This assumes 4 layers per volume (except last volume)
    layer = (barrel_hits.layid / 2 - 1 + volume * 4).astype(np.int8)
    return (barrel_hits[['evtid', 'barcode', 'phi', 'z']]
            .assign(volume=volume, layer=layer))

def select_signal_hits(hits):
    """Select signal hits from tracks that hit all barrel layers"""
    return (hits.groupby(['evtid', 'barcode'])
            # Select tracks that hit every layer at least once
            .filter(lambda x: len(x) >= 10 and x.layer.unique().size == 10)
            # Average duplicate hits together
            .groupby(['evtid', 'barcode', 'layer'], as_index=False).mean())

## Read the data

In [4]:
data_dir = '/global/cscratch1/sd/sfarrell/ACTS/prod_mu10_pt1000_2017_07_29'
#data_dir = '/bigdata/shared/ACTS/prod_mu10_pt1000_2017_07_29'

In [5]:
n_files = 1

all_files = os.listdir(data_dir)
hits_files = sorted(f for f in all_files if f.startswith('clusters'))
hits_files = [os.path.join(data_dir, f) for f in hits_files[:n_files]]

In [6]:
%%time

n_workers = 1
hits = process_files(hits_files, num_workers=n_workers)

Loading /global/cscratch1/sd/sfarrell/ACTS/prod_mu10_pt1000_2017_07_29/clusters_1.csv
CPU times: user 39.5 ms, sys: 48.1 ms, total: 87.6 ms
Wall time: 1min 4s


In [7]:
print('Hits data shape:', hits.shape)

Hits data shape: (520540, 7)


In [8]:
%%time

# Select hits
selected_hits = select_hits(hits)
print('selected barrel hits:', selected_hits.shape)
signal_hits = select_signal_hits(selected_hits)
print('signal track hits:', signal_hits.shape)

selected barrel hits: (327771, 6)
signal track hits: (134990, 6)
CPU times: user 9.01 s, sys: 34.8 ms, total: 9.05 s
Wall time: 9.09 s


In [9]:
%%time

# Gather features into tensor of shape (events, layers, features)
input_data = np.stack(signal_hits.groupby(['evtid', 'barcode'])
                      .apply(lambda x: x[['phi', 'z', 'layer']].values)).astype(np.float32)

# Scale coordinates to approx [-1, 1] for phi and z, [0, 1] for layer number
coord_scale = np.array([np.pi, 1000., 10.])
input_data[:,:] /= coord_scale

CPU times: user 6min 10s, sys: 1.51 s, total: 6min 12s
Wall time: 6min 11s


## Define the model

We define an LSTM model in PyTorch which will predict the next hit location.

In [10]:
class TrackFilterer(nn.Module):

    def __init__(self, input_dim=3, hidden_dim=5, output_dim=2, n_lstm_layers=1):
        super(TrackFilterer, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_lstm_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        input_size = x.size()
        # Initialize the lstm hidden state
        h = (torch_zeros(self.lstm.num_layers, input_size[0], self.lstm.hidden_size),
             torch_zeros(self.lstm.num_layers, input_size[0], self.lstm.hidden_size))
        x, h = self.lstm(x, h)
        # Flatten layer axis into batch axis so FC applies independently across layers.
        x = (self.fc(x.contiguous().view(-1, x.size(-1)))
             .view(input_size[0], input_size[1], -1))
        return x

## Configure and train the model

In [11]:
# Model config
hidden_dim = 20
n_lstm_layers = 1

# Train config
n_epochs = 200
batch_size = 32
test_frac = 0.1

In [12]:
# Split data into training, validation, and test sets
train_data, test_data = train_test_split(input_data, test_size=test_frac)

# Inputs are the hits from [0, N-1)
# Targets are the hits from [1, N) without the layer feature.
train_input = np_to_torch(train_data[:,:-1])
train_target = np_to_torch(train_data[:,1:,:2])
test_input = np_to_torch(test_data[:,:-1])
test_target = np_to_torch(test_data[:,1:,:2])

n_samples = train_input.size(0)
n_batches = (n_samples + batch_size - 1) // batch_size
print('Training samples:', n_samples)
print('Batches per epoch:', n_batches)
print('Test samples:', test_input.size(0))

Training samples: 615537
Batches per epoch: 19236
Test samples: 68394


In [ ]:
# Construct the model and estimator
estimator = Estimator(
    TrackFilterer(hidden_dim=hidden_dim),
    loss_func=nn.MSELoss(), cuda=cuda)

TrackFilterer (
  (lstm): LSTM(3, 20, batch_first=True)
  (fc): Linear (20 -> 2)
)
Parameters: 2042


In [ ]:
%%time

# Train the model
estimator.fit(train_input, train_target,
              valid_input=test_input, valid_target=test_target,
              batch_size=batch_size, n_epochs=n_epochs)

Training samples: 615537
Batches per epoch: 19236
Validation samples: 68394
Epoch 0
  training loss 0.00662 time 157.933s
  validate loss 0.00627
Epoch 2
  training loss 0.00601 time 163.919s
  validate loss 0.00547
Epoch 3
  training loss 0.00544 time 163.744s
  validate loss 0.00492
Epoch 4
  training loss 0.00502 time 159.187s
  validate loss 0.00455
Epoch 5
  training loss 0.00465 time 158.514s
  validate loss 0.00436
Epoch 6
  training loss 0.00433 time 165.388s
  validate loss 0.0043
Epoch 7
  training loss 0.00407 time 162.893s
  validate loss 0.00415
Epoch 8
  training loss 0.00387 time 166.608s
  validate loss 0.00399
Epoch 9
  training loss 0.00367 time 165.504s
  validate loss 0.00351
Epoch 11
  training loss 0.00347 time 166.627s
  validate loss 0.00337
Epoch 12
  training loss 0.00328 time 166.973s
  validate loss 0.00323
Epoch 13
  training loss 0.00317 time 166.277s
  validate loss 0.00312
Epoch 14
  training loss 0.00308 time 166.048s
  validate loss 0.00302
Epoch 15
  

In [ ]:
plt.figure()
epochs = np.arange(n_epochs)
plt.semilogy(epochs, np.array(estimator.train_losses), label='Training set')
plt.semilogy(epochs, np.array(estimator.valid_losses), label='Validation set')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training loss')
plt.legend(loc=0)
plt.tight_layout()

## Evaluate model performance

In [ ]:
# Get the predictions and errors for the full training set
train_output = estimator.model(train_input)
train_error = train_output - train_target
train_resid = train_error.cpu().data.numpy() * coord_scale[:2]

# Get the predictions and errors for the full test set
test_output = estimator.model(test_input)
test_error = test_output - test_target
test_resid = test_error.cpu().data.numpy() * coord_scale[:2]

In [ ]:
plt.figure(figsize=(9,4))
plt.subplot(121)
hist_args = dict(bins=50, range=(-0.2, 0.2), normed=True, log=False, histtype='step')
plt.hist(train_resid[:,:,0].flatten(), label='Training set', **hist_args)
plt.hist(test_resid[:,:,0].flatten(), label='Test set', **hist_args)
plt.xlabel('Error in $\phi$ [rad]')
plt.ylabel('Normalized sample counts')
plt.legend(loc=0)

plt.subplot(122)
hist_args = dict(bins=50, range=(-100, 100), normed=True, log=False, histtype='step')
plt.hist(train_resid[:,:,1].flatten(), label='Training set', **hist_args)
plt.hist(test_resid[:,:,1].flatten(), label='Test set', **hist_args)
plt.xlabel('Error in z [mm]')
plt.ylabel('Normalized sample counts')
plt.legend(loc=0)
plt.tight_layout()

#### Observations

- The $\phi$ residual has a funny asymmetric shape. Not sure what's going on there.
- Training and test set agree perfectly.
- Non-gaussian tails in the Z residual.

## Visualize trajectories

In [ ]:
for i in range(6):
    layers = np.arange(10)
    inputs = test_input[i].cpu().data.numpy() * coord_scale
    outputs = test_output[i].cpu().data.numpy() * coord_scale[:2]
    targets = test_target[i].cpu().data.numpy() * coord_scale[:2]

    plt.figure(figsize=(9,3))

    plt.subplot(121)
    plt.plot(layers[:-1], inputs[:,0], 'b.-')
    plt.plot(layers[1:], targets[:,0], 'b.-', label='Data')
    plt.plot(layers[1:], outputs[:,0], 'r.-', label='Filter')
    plt.xlabel('Detector layer')
    plt.ylabel('$\phi$ [rad]')
    plt.legend(loc=0)

    plt.subplot(122)
    plt.plot(layers[:-1], inputs[:,1], 'b.-')
    plt.plot(layers[1:], targets[:,1], 'b.-', label='Data')
    plt.plot(layers[1:], outputs[:,1], 'r.-', label='Filter')
    plt.xlabel('Detector layer')
    plt.ylabel('z [mm]')
    plt.legend(loc=0)

    plt.tight_layout()

#### Observations

- Wrap-around at $\phi = \pi$ can screw up the filter (not currently shown).
- Filter is often smoother than data in the coarse outer layers.
- Few examples seem a bit unstable.